In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd

In [2]:
def read_images():
    training_images = []
    i = 0
    DIR = "E:\Galaxy Zoo\Data\images_training_rev1\images_training_rev1"
    for img in os.listdir(DIR):
        im = cv2.imread(os.path.join(DIR,img))
        im = im[106:106*3,106:106*3,:]          #cropping 424x424->212x212
        im = cv2.resize(im,(106,106), interpolation=cv2.INTER_CUBIC)   #downsampling to half resolution 106x106
        training_images.append(im)
        i+=1
        if i==10000:
            break
        
    return training_images
    

In [3]:
training_images = read_images()
training_images = np.array(training_images).reshape(-1,106,106,3)

In [4]:
X_train = training_images[:8000]
x_test = training_images[8000:]
df = pd.read_csv("training_solutions_rev1/training_solutions_rev1.csv")
y_train = df.drop('GalaxyID', axis = 1).values
y_train = y_train[:10000]
y_test = y_train[8000:10000]
y_train = y_train[:8000]

In [5]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,Lambda
from keras.optimizers import SGD, RMSprop, Adam

Using TensorFlow backend.


In [8]:
def Conv(model,layers,filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1))) 
        model.add(Conv2D((filters), kernel_size = (3,3)))
        model.add(Activation("relu"))
        
    model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))

In [9]:
model = Sequential()
model.add(Lambda(lambda x : x, input_shape=(106,106,3)))

Conv(model,2,64)
Conv(model,2,128)
Conv(model,3,256)
Conv(model,3,512)
Conv(model,3,512)

model.add(Flatten())
model.add(Dense(4096))
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(Dense(4096))
model.add(Activation("relu"))
model.add(Dropout(0.3))


model.add(Dense(37))
model.add(Activation("sigmoid"))

model.compile(optimizer = RMSprop(lr=1e-6), loss='mean_squared_error', metrics = ['accuracy'])


W0802 23:24:07.640994 11072 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0802 23:24:07.880958 11072 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0802 23:24:07.892992 11072 deprecation.py:506] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0802 23:24:07.980997 11072 d

In [10]:
print(y_train.shape,
X_train.shape)

(8000, 37) (8000, 106, 106, 3)


In [11]:
model.fit(X_train,y_train, epochs = 5)

W0802 23:25:11.675883 11072 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
8000/8000 [==============================] - 1103s 138ms/step - loss: 0.0530 - acc: 0.5245
Epoch 2/5
8000/8000 [==============================] - 1092s 136ms/step - loss: 0.0294 - acc: 0.5851
Epoch 3/5
8000/8000 [==============================] - 1044s 130ms/step - loss: 0.0271 - acc: 0.5906
Epoch 4/5
8000/8000 [==============================] - 1030s 129ms/step - loss: 0.0253 - acc: 0.5830
Epoch 5/5
8000/8000 [==============================] - 902s 113ms/step - loss: 0.0242 - acc: 0.5793


In [12]:
model.evaluate(x_test,y_test)

2000/2000 [==============================] - 48s 24ms/step


[0.02201651184260845, 0.601]

In [13]:
model.save("20819.model")